In [25]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [26]:
# 라이브러리 가져오기
import requests
import pandas as pd
import os
import dotenv 

In [27]:
dotenv.load_dotenv()
my_key = os.getenv('KAKAO_RESTFULL_KEY')

In [28]:
# Kakao Map API 통해 위도, 경도 데이터 가져오는 함수를 정의
def get_geocoding(place):
    try:
        url = f"https://dapi.kakao.com/v2/local/search/address"
        headers = {"Authorization": f"KakaoAK {my_key}"}
        params = {
            'query': place
        }
        response = requests.get(url, headers=headers, params=params)
        result = response.json()
        if result['documents']:
            return result["documents"][0]["y"], result["documents"][0]["x"]
        else:
            return None, None
    except:
        return pd.notna() # 결측치 (데이터 없음 또는 유효하지 않은 데이터)

In [29]:
lat = []  #위도
lng = []  #경도

# 장소(주소) 리스트
places = ["서울특별시 종로구 세종대로 175", 
          "서울특별시 서초구 서초동 700", 
          "부산광역시 해운대구 해운대해변로 264"]

i=0
for place in places:   
    i = i + 1
    try:
        print(i, place)
        # get_geocoding 함수의 리턴값 호출하여 geo_location 변수에 저장
        place_lat, place_lon = get_geocoding(place)
        lat.append(place_lat)
        lng.append(place_lon)
        
    except:
        lat.append('')
        lng.append('')

# 데이터프레임으로 변환하기
df = pd.DataFrame({'위도':lat, '경도':lng}, index=places)

df

1 서울특별시 종로구 세종대로 175
2 서울특별시 서초구 서초동 700
3 부산광역시 해운대구 해운대해변로 264


,위도,경도
서울특별시 종로구 세종대로 175,37.5718478584908,126.976168275947
서울특별시 서초구 서초동 700,37.4810862955299,127.015245160054
부산광역시 해운대구 해운대해변로 264,35.1591069824231,129.160283786856


In [30]:
df = pd.read_csv('https://raw.githubusercontent.com/pia222sk20/python/main/data/sample.csv', encoding='cp949')
df.head()
df_simple = df.loc[:,['상호명', '도로명주소']]
df_simple.head()

,상호명,도로명주소
0,하나산부인과,경기도 안산시 단원구 달미로 10
1,타워광명내과의원,서울특별시 강남구 언주로30길 39
2,조정현신경외과의원,경기도 시흥시 중심상가로 178
3,한귀원정신과의원,부산광역시 수영구 수영로 688
4,더블유스토어수지점,경기도 용인시 수지구 문정로 32


In [31]:
df_simple['도로명주소'].index

RangeIndex(start=0, stop=91335, step=1)

In [ ]:
df_simple = df_simple.dropna() # 데이터 중에서 None 또는 NaN 없는 데이터 row를 삭제 

In [35]:
import random
random_index = random.sample(range(91135), 50)

In [38]:
df_simple_1000 = df_simple.iloc[random_index]
results = [ get_geocoding(addr) for addr in df_simple_1000['도로명주소'].values]
len(results)

50

In [48]:
df_simple_1000.loc[:,'위도'], df_simple_1000.loc[:,'경도'] = zip(*results)

In [ ]:
# 지도
%pip install folium

In [55]:
import folium

m = folium.Map(location=(35, 130)) # 중심좌표
for idx, row in df_simple_1000.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['상호명'],
        tooltip=['상호명'][-2:]
    ).add_to(m)
m.save('map.html')
m

In [51]:
for idx, row in df_simple_1000.iterrows():
    print(idx, row['위도'], row['경도'])
    break

22298 37.5221332556567 126.87778960592
